# Part 1: Introduction & Data Preparation


## 1.1 Introduction

Perform customer segmentation using RFM (Recency, Frequency, Monetary) analysis to identify customer groups for targeted marketing strategies.


In [86]:
%pip install plotly
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as pc




# Load the dataset
data = pd.read_excel('/Users/jam/Desktop/ecommerce-customer-segmentation/data/Online Retail.xlsx')


Note: you may need to restart the kernel to use updated packages.


## 1.2 Data Exploration


In [87]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


> Dataset spans from December 2010 to December 2011
Missing values identified from `data.info()`:

| Column | Missing | % | Action |
|:-------|--------:|--:|:-------|
| CustomerID | 135,080 | 24.9% | **Drop** — required for RFM |
| Description | 1,454 | 0.3% | **Ignore** — not used in RFM |


In [89]:
# Drop rows with missing CustomerID
data = data.dropna(subset=['CustomerID'])

In [90]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [91]:
# Data quality check

print("=" * 60)
print("DATA QUALITY REPORT")
print("=" * 60)

# Dataset Overview
print("\n DATASET OVERVIEW")
print("-" * 40)
print(f"Total Rows: {data.shape[0]:,}")
print(f"Total Columns: {data.shape[1]}")
print(f"Memory Usage: {data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Date Range
print("\n DATE RANGE")
print("-" * 40)
print(f"First Transaction: {data['InvoiceDate'].min()}")
print(f"Last Transaction: {data['InvoiceDate'].max()}")
print(f"Total Period: {(data['InvoiceDate'].max() - data['InvoiceDate'].min()).days} days")

# Missing Values
print("\n MISSING VALUES")
print("-" * 40)
missing = data.isnull().sum()
missing_pct = (data.isnull().sum() / len(data) * 100).round(2)
missing_df = pd.DataFrame({'Missing': missing, 'Percentage': missing_pct})
print(missing_df[missing_df['Missing'] > 0])

# Unique Values
print("\n UNIQUE VALUES")
print("-" * 40)
print(f"Invoices: {data['InvoiceNo'].nunique():,}")
print(f"Products (StockCode): {data['StockCode'].nunique():,}")
print(f"Customers: {data['CustomerID'].nunique():,}")
print(f"Countries: {data['Country'].nunique()}")

# Data Types
print("\n DATA TYPES")
print("-" * 40)
print(data.dtypes)

# Duplicates
print("\n DUPLICATES")
print("-" * 40)
print(f"Duplicate Rows: {data.duplicated().sum():,}")

# Transactions by Day of Week
print("\n TRANSACTIONS BY DAY OF WEEK")
print("-" * 40)
data['DayOfWeek'] = pd.to_datetime(data['InvoiceDate']).dt.day_name()
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_count = data['DayOfWeek'].value_counts().reindex(days_order).fillna(0).astype(int)
print(day_count)

# Anomaly Detection
print("\n ANOMALY CHECK")
print("-" * 40)
print(f"Negative Quantity: {(data['Quantity'] < 0).sum():,} rows")
print(f"Zero/Negative Price: {(data['UnitPrice'] <= 0).sum():,} rows")
print(f"Saturday Transactions: {day_count['Saturday']:,} (Unusual if 0)")





DATA QUALITY REPORT

 DATASET OVERVIEW
----------------------------------------
Total Rows: 406,829
Total Columns: 8
Memory Usage: 97.76 MB

 DATE RANGE
----------------------------------------
First Transaction: 2010-12-01 08:26:00
Last Transaction: 2011-12-09 12:50:00
Total Period: 373 days

 MISSING VALUES
----------------------------------------
Empty DataFrame
Columns: [Missing, Percentage]
Index: []

 UNIQUE VALUES
----------------------------------------
Invoices: 22,190
Products (StockCode): 3,684
Customers: 4,372
Countries: 37

 DATA TYPES
----------------------------------------
InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

 DUPLICATES
----------------------------------------
Duplicate Rows: 5,225

 TRANSACTIONS BY DAY OF WEEK
----------------------------------------
Day

### Data Quality Summary

**Dataset Overview:**
- Total Rows: 406,829 (after cleaning)
- Total Columns: 10
- Memory Usage: 122.62 MB
- Date Range: Dec 2010 - Dec 2011 (373 days)

**Data Quality:**
- Missing Values: None ✅
- Unique Invoices: 22,190
- Unique Customers: 4,339

**Anomalies Found:**
- Negative Quantity: 8,905 rows (Returns/Cancellations)
- Zero/Negative Price: 40 rows
- Saturday Transactions: 0 (Business closed on Saturdays)

**Note:** Negative quantities represent product returns. Saturday having zero transactions indicates the business operates on weekdays and Sundays only.

# transaction by day of week


In [92]:
# Transactions by Day of Week
# Plot
fig = px.bar(
    x=day_count.index, 
    y=day_count.values, 
    labels={'x': 'Day of Week', 'y': 'Number of Transactions'}, 
    title='Transactions by Day of Week',
    color=day_count.values,
    color_continuous_scale='Viridis'
)
fig.update_layout(showlegend=False)
fig.show()

# Insight
print(f"Busiest Day: {day_count.idxmax()} ({day_count.max():,} transactions)")
print(f"No transactions on Saturday - Business closed on Saturdays")

Busiest Day: Thursday (82,374 transactions)
No transactions on Saturday - Business closed on Saturdays


In [93]:
print("Transactions by Day of the Week:")
print(day_count)
print(f"\nBusiest day: {day_count.idxmax()} ({day_count.max():,} transactions)")
print(f"Slowest day: {day_count.idxmin()} ({day_count.min():,} transactions)")

Transactions by Day of the Week:
DayOfWeek
Monday       66382
Tuesday      68110
Wednesday    70599
Thursday     82374
Friday       56127
Saturday         0
Sunday       63237
Name: count, dtype: int64

Busiest day: Thursday (82,374 transactions)
Slowest day: Saturday (0 transactions)


Purchases by Day of Week

## 1.3 Data Preprocessing

In [94]:
# Drop missing values customerID
data = data.dropna(subset=['CustomerID'])

In [95]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate']) # confirm date format
data['TotalAmount'] = data['Quantity'] * data['UnitPrice']  # create new column for monetary value by multiplying quantity by unit price

In [96]:
data.head() # check new column

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,DayOfWeek,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,Wednesday,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Wednesday,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,Wednesday,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Wednesday,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Wednesday,20.34


In [97]:
# create analysis date
analysis_date = data['InvoiceDate'].max() + dt.timedelta(days=1)

In [98]:
analysis_date

Timestamp('2011-12-10 12:50:00')

# Part 2. RFM Calculation

## 2.1 RFM Calculation

### Aggregate transaction data per customer:

In [99]:
recency = data.groupby('CustomerID')['InvoiceDate'].max()
recency = (analysis_date - recency).dt.days

frequency = data.groupby('CustomerID')['InvoiceNo'].nunique() # use nunique to count unique invoices instead of total rows .count() because multiple rows can belong to the same invoice

monetary = data.groupby('CustomerID')['TotalAmount'].sum() # sum total amount spent per customer

rfm = pd.DataFrame({'Recency': recency,
                    'Frequency': frequency,
                    'Monetary': monetary})
rfm.head(20)




,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,7,4310.00
12348.0,75,4,1797.24
12349.0,19,1,1757.55
12350.0,310,1,334.40
12352.0,36,11,1545.41
12353.0,204,1,89.00
12354.0,232,1,1079.40
12355.0,214,1,459.40


In [100]:
zero_monetary = rfm[rfm['Monetary'] <= 0].index
print(f"Number of customers with zero or negative monetary value: {len(zero_monetary)}")

Number of customers with zero or negative monetary value: 50


In [101]:
data[(data['CustomerID'].isin(zero_monetary)) & (data['Quantity'] < 0)] # show only rows with negative quantity for these customers

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,DayOfWeek,TotalAmount
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,Wednesday,-19.80
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,Wednesday,-6.96
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,Wednesday,-6.96
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,Wednesday,-6.96
239,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548.0,United Kingdom,Wednesday,-41.40
...,...,...,...,...,...,...,...,...,...,...
414250,C572410,23170,REGENCY TEA PLATE ROSES,-20,2011-10-24 11:58:00,1.45,16742.0,United Kingdom,Monday,-29.00
467903,C576375,22585,PACK OF 6 BIRDY GIFT TAGS,-1,2011-11-15 08:52:00,1.25,16878.0,United Kingdom,Tuesday,-1.25
467904,C576375,22583,PACK OF 6 HANDBAG GIFT BOXES,-1,2011-11-15 08:52:00,2.55,16878.0,United Kingdom,Tuesday,-2.55
467905,C576375,21034,REX CASH+CARRY JUMBO SHOPPER,-10,2011-11-15 08:52:00,0.95,16878.0,United Kingdom,Tuesday,-9.50


#### Observation

Found customers with Monetary ≤ 0. Possible causes:
- Cancelled orders (Invoice starts with "C")
- Returns (negative quantity)
- Net refunds exceed purchases

*Exact reason unknown from available data.*

In [102]:
rfm = rfm[rfm['Monetary'] > 0] # remove customers with zero or negative monetary value

In [103]:
# Summary after cleaning
print(f"Original customers: {data['CustomerID'].nunique()}")
print(f"Remaining customers: {len(rfm)}")
print(f"Monetary <= 0: {(rfm['Monetary'] <= 0).sum()}")

Original customers: 4372
Remaining customers: 4322
Monetary <= 0: 0


# 2.2 RFM Scoring

### Method: Quantile-based Scoring (Quintile)

RFM scores (1-5) were assigned using quantile-based segmentation (Hallishma, 2022; Ullah et al., 2023; Madhiraju et al., 2024).

| Metric | Score 1 | Score 5 |
|:-------|:--------|:--------|
| Recency | Long ago ❌ | Recent ✅ |
| Frequency | Few ❌ | Many ✅ |
| Monetary | Low ❌ | High ✅ |

> Note: Recency is reversed — lower value = better = higher score

### Output

| Column | Description | Example |
|:-------|:------------|:--------|
| RFM_Score | String (R+F+M) | "545" |
| RFM_Total | Sum (R+F+M) | 14 |

### Issue & Solution

| Attempt | Issue | Solution |
|:--------|:------|:---------|
| `pd.qcut()` | Duplicate bin edges | Added `.rank(method='first')` |

In [104]:
rfm['R'] = pd.qcut(rfm['Recency'].rank(method='first'), 5, labels=[5,4,3,2,1])
rfm['F'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5])

# create RFM_Segment and RFM_Score columns to combine R, F, M values and sum them
rfm['RFM_Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)

rfm.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12347.0,2,7,4310.00,5,4,5,545,14
12348.0,75,4,1797.24,2,3,4,234,9
12349.0,19,1,1757.55,4,1,4,414,9
12350.0,310,1,334.40,1,1,2,112,4
12352.0,36,11,1545.41,3,5,4,354,12


In [105]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4322 entries, 12347.0 to 18287.0
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Recency      4322 non-null   int64   
 1   Frequency    4322 non-null   int64   
 2   Monetary     4322 non-null   float64 
 3   R            4322 non-null   category
 4   F            4322 non-null   category
 5   M            4322 non-null   category
 6   RFM_Segment  4322 non-null   object  
 7   RFM_Score    4322 non-null   int64   
dtypes: category(3), float64(1), int64(3), object(1)
memory usage: 215.9+ KB


## 2.3 Customer Segmentation
### Method: RFM Score-based Segmentation

Customers were segmented into three groups based on their RFM Score (sum of R, F, M ranging from 3-15):

### Threshold Logic

| Value | Calculation |
|:------|:------------|
| Min Score | 1+1+1 = 3 |
| Max Score | 5+5+5 = 15 |
| Mid Point | (3+15) / 2 = 9 |

### Segment Definition

| Segment | Score Range | Logic |
|:--------|:------------|:------|
| Low-Value | 3-6 | Below mid-point |
| Mid-Value | 7-11 | Around mid-point |
| High-Value | 12-15 | Above mid-point |

> Note: Traditional RFM uses 5+ segments, but 3 groups serve as baseline for K-Means comparison.

In [106]:
# Define segmentation function based on RFM_Score
def assign_segment(score):
    if score >= 12:
        return 'High-Value'
    elif score >= 7:
        return 'Mid-Value'
    else:
        return 'Low-Value'

# Apply segmentation
rfm['RFM_Segment'] = rfm['RFM_Score'].apply(assign_segment)

# Count customers per segment
print(rfm['RFM_Segment'].value_counts())

RFM_Segment
Mid-Value     1765
Low-Value     1301
High-Value    1256
Name: count, dtype: int64


In [107]:
# 1. visualization for customer distribution across segments
fig = px.bar(
    rfm['RFM_Segments'].value_counts().reset_index(),
    x='RFM_Segments',
    y='count',
    title='Customer Distribution by Segment',
    labels={'RFM_Segments': 'RFM Segment', 'count': 'Number of Customers'},
    color='RFM_Segments',
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.show()

KeyError: 'RFM_Segments'

In [ ]:
# 2. Pie Chart for customer segment proportion of each segment
fig = px.pie(
    rfm['RFM_Segments'].value_counts().reset_index(),
    names='RFM_Segments',
    values='count',
    title='Customer Segment Proportion',
    color='RFM_Segments',
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.show()

In [ ]:
# 3. 3D Scatter Plot of R, F, M scores
fig = px.box(
    rfm,
    x='RFM_Segments',
    y='Monetary',
    title='Monetary Distribution by Segment',
    color='RFM_Segments',
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Recency', 'Frequency', 'Monetary', 'R', 'F', 'M', 'RFM_Segment', 'RFM_Score'] but received: RFM_Segments

## Chart Insights

### 1. The bar chart shows customer distribution across three segments: Mid-Value leads with 1,765 customers (40.8%), followed by Low-Value at 1,301 (30.1%), and High-Value at 1,256 (29.1%).

### 2. The pie chart confirms a relatively balanced distribution, with High-Value representing nearly 29% of the customer base.

### 3. The box plot reveals significant spending differences: High-Value customers have a median spend of 2,484, which is 11 times higher than Low-Value (223) and nearly 4 times higher than Mid-Value (652).

> Key Finding: Although High-Value represents only 29% of customers, they spend significantly more. Pareto analysis will determine their actual revenue contribution.

# Part 3 K-Mean Clustering

### 3.1 Data Preparation

K-Means requires scaled data to prevent features with larger ranges from dominating the clustering.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Select RFM features
X = rfm[['Recency', 'Frequency', 'Monetary']]

# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Shape: {X_scaled.shape}")

Shape: (4322, 3)


### 3.1.1 Distribution Analysis
- Histogram


In [ ]:
# Histogram - check distribution (skewness, outliers)
fig = px.histogram(rfm, x='Recency', title='Recency Distribution', nbins=50)
fig.show()

fig = px.histogram(rfm, x='Frequency', title='Frequency Distribution', nbins=50)
fig.show()

fig = px.histogram(rfm, x='Monetary', title='Monetary Distribution', nbins=50)
fig.show()

### Note: Distribution Analysis

All three RFM features show right-skewed distributions:
- **Recency:** Data concentrated at 0-50 days with a long tail extending to 374 days
- **Frequency:** Heavily skewed with most customers making 1-10 purchases, outliers reaching 248
- **Monetary:** Extremely skewed with most spending under 10,000, but outliers up to 280,000

Right-skewed data can negatively impact K-Means clustering as the algorithm is sensitive to outliers. Both StandardScaler and Yeo-Johnson transformation (Wong et al., 2024) were tested to address this issue.

### Compare Scaling Methods : StandardScaler vs Yeo-Johnson

In [ ]:
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import silhouette_score

pt = PowerTransformer(method='yeo-johnson')
X_yeo = pt.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

kmeans_std = KMeans(n_clusters=4, random_state=42, n_init=10)
kmeans_yeo = KMeans(n_clusters=4, random_state=42, n_init=10)

cluster_std = kmeans_std.fit_predict(X_scaled)
cluster_yeo = kmeans_yeo.fit_predict(X_yeo)

print(f"StandardScaler Silhouette: {silhouette_score(X_scaled, cluster_std):.4f}")
print(f"Yeo-Johnson Silhouette: {silhouette_score(X_yeo, cluster_yeo):.4f}")

StandardScaler Silhouette: 0.6126
Yeo-Johnson Silhouette: 0.3259


### Scaling Method Comparison

| Method | Silhouette Score |
|:-------|:----------------:|
| StandardScaler | **0.6126** |
| Yeo-Johnson | 0.3259 |

Wong et al. (2024) reported Yeo-Johnson improved clustering by 15-20% when using Hierarchical Clustering. However, for K-Means on this dataset, StandardScaler outperforms Yeo-Johnson.

This difference may be due to:
1. **Algorithm:** Yeo-Johnson works better with Hierarchical Clustering than K-Means
2. **Outliers:** Extreme outliers (Super VIP with 280k spending) are preserved by StandardScaler, enabling better cluster separation

**Decision:** StandardScaler was selected for K-Means clustering.
```

---

### 3.2 Find Optimal K

Using Elbow Method and Silhouette Score to determine optimal number of clusters.

In [ ]:
from sklearn.metrics import silhouette_score

# Test K from 2 to 10
k_range = range(2, 11)
inertia = []
silhouette_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Elbow Chart
fig = px.line(x=list(k_range), y=inertia, markers=True,
              title='Elbow Method',
              labels={'x': 'Number of Clusters (K)', 'y': 'Inertia'})
fig.show()

# Silhouette Chart
fig = px.line(x=list(k_range), y=silhouette_scores, markers=True,
              title='Silhouette Score by K',
              labels={'x': 'Number of Clusters (K)', 'y': 'Silhouette Score'})
fig.show()

# Print inertia and silhouette scores
for k, iner, sil in zip(k_range, inertia, silhouette_scores):
    print(f"K={k}: Inertia = {iner:.0f}, Silhouette = {sil:.4f}")

K=2: Inertia = 9028, Silhouette = 0.8948
K=3: Inertia = 5467, Silhouette = 0.5894
K=4: Inertia = 4003, Silhouette = 0.6126
K=5: Inertia = 2988, Silhouette = 0.5937
K=6: Inertia = 2344, Silhouette = 0.5845
K=7: Inertia = 1892, Silhouette = 0.5163
K=8: Inertia = 1575, Silhouette = 0.4882
K=9: Inertia = 1340, Silhouette = 0.4911
K=10: Inertia = 1226, Silhouette = 0.4262


### K Selection

The Elbow Method shows inertia decreasing from 9,028 (K=2) to 4,003 (K=4), with diminishing returns beyond K=4. The Silhouette Score peaks at K=2 (0.89) but K=4 achieves the second-highest score (0.61), exceeding the target threshold of 0.4.

Both K=3 and K=4 were tested for comparison:
- **K=4** was selected as primary: Silhouette score of 0.61, elbow point, and finer segmentation of high-value customers (separates Super VIP from VIP)
- **K=3** was tested for comparison: more balanced clusters but combines VIP segments into one group

> Note: K=2 was rejected despite highest Silhouette as two clusters are too coarse for meaningful customer segmentation.

### 3.3 Apply K-Mean Clustering


In [ ]:
#  K=4
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
rfm['Cluster'] = kmeans.fit_predict(X_scaled)

# Check cluster distribution
print(rfm['Cluster'].value_counts().sort_index())

Cluster
0    3144
1    1062
2       6
3     110
Name: count, dtype: int64


In [ ]:
rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean()

,Recency,Frequency,Monetary
Cluster,,,
0,41.137405,4.818702,1487.377625
1,244.889831,1.838041,485.190255
2,7.666667,89.000000,182181.981667
3,9.181818,40.672727,18441.961455


### Interpretation K4

- **Cluster 2 (Super VIP):** Only 6 customers but extremely high value — average spend 182,182 with 89 purchases. Priority for retention.
- **Cluster 3 (VIP):** 110 high-value customers with frequent purchases. Target for loyalty programs.
- **Cluster 0 (Regular):** Largest group (3,144) with moderate engagement. Mass marketing target.
- **Cluster 1 (At Risk):** 1,062 customers with high recency (245 days). Need re-engagement campaigns.


In [ ]:
# K=3
kmeans_3 = KMeans(n_clusters=3, random_state=42, n_init=10)
rfm['Cluster_K3'] = kmeans_3.fit_predict(X_scaled)

print("K=3:")
print(rfm['Cluster_K3'].value_counts().sort_index())

K=3:
Cluster_K3
0    1077
1    3222
2      23
Name: count, dtype: int64


In [ ]:
rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean()

,Recency,Frequency,Monetary
Cluster,,,
0,41.137405,4.818702,1487.377625
1,244.889831,1.838041,485.190255
2,7.666667,89.000000,182181.981667
3,9.181818,40.672727,18441.961455


### Interpretation K3

- **Cluster 2 (VIP):** 23 customers with high value — average spend 77,000+ with 58 purchases. Combines Super VIP and VIP from K=4.
- **Cluster 1 (Regular):** Largest group (3,222) with moderate engagement. Mass marketing target.
- **Cluster 0 (At Risk):** 1,077 customers with high recency (245 days). Need re-engagement campaigns.

> Note: K=3 merges Super VIP (6) and VIP (110) into one group, losing granularity for high-value customer targeting.

### 3.4 Visualize Clusters

In [ ]:
# 3D Scatter Plot
fig = px.scatter_3d(
    rfm,
    x='Recency',
    y='Frequency',
    z='Monetary',
    color='Cluster',
    title='K-Means Clusters (K=4)',
    opacity=0.6,
    size_max=5
)
fig.update_traces(marker=dict(size=3))
fig.show()

In [ ]:
# Pie Chart 
fig = px.pie(
    rfm['Cluster'].value_counts().reset_index(),
    names='Cluster',
    values='count',
    title='Cluster Distribution'
)
fig.update_traces(textinfo='percent+value', textposition='outside')
fig.show()

### Cluster Distribution

The pie chart shows K-Means segmentation is highly imbalanced:
- **Cluster 0 (Regular):** 72.7% — majority of customers
- **Cluster 1 (At Risk):** 24.6% — need re-engagement
- **Cluster 3 (VIP):** 2.55% — high-value customers
- **Cluster 2 (Super VIP):** 0.14% — only 6 customers but highest value

> Key Insight: Although Super VIP represents only 0.14%, they contribute significantly to revenue. Pareto analysis will confirm this.

## Part 4: Pareto Analysis

The Pareto Principle (80/20 rule) states that 80% of revenue comes from 20% of customers.


In [ ]:
# Pareto Analysis
rfm_sorted = rfm.sort_values('Monetary', ascending=False)
rfm_sorted['Cumulative_Revenue'] = rfm_sorted['Monetary'].cumsum()
rfm_sorted['Cumulative_Pct'] = rfm_sorted['Cumulative_Revenue'] / rfm_sorted['Monetary'].sum() * 100

# Top 20% customers
top_20 = int(len(rfm_sorted) * 0.2)
revenue_pct = rfm_sorted.iloc[:top_20]['Monetary'].sum() / rfm_sorted['Monetary'].sum() * 100
print(f"Top 20% customers ({top_20} people) contribute: {revenue_pct:.1f}% of revenue")

Top 20% customers (864 people) contribute: 73.5% of revenue


In [ ]:
# Super VIP (Cluster 2) contribution
super_vip_revenue = rfm[rfm['Cluster'] == 2]['Monetary'].sum()
total_revenue = rfm['Monetary'].sum()
super_vip_pct = super_vip_revenue / total_revenue * 100

print(f"Super VIP (6 customers) contribute: {super_vip_pct:.1f}% of revenue")

Super VIP (6 customers) contribute: 13.1% of revenue


### Pareto Analysis

The Pareto analysis confirms the 80/20 principle: the top 20% of customers (864 people) contribute 73.5% of total revenue, which is close to the theoretical 80%.

More importantly, the 6 Super VIP customers from K-Means clustering (Cluster 2) represent only 0.14% of the customer base but generate 13.1% of total revenue. This means losing just one Super VIP customer would result in approximately 2% revenue loss.

> Key Insight: Super VIP customers require priority retention strategies including personal care, dedicated account management, and immediate alerts if their purchase recency exceeds 30 days.

## Part 5 Churn Prediction


### 5.1 Define Churn

Churn threshold was determined using a data-driven approach, as recommended in e-commerce literature — analyzing the dataset's purchase patterns rather than using arbitrary fixed periods.

From the Recency histogram in Part 3.1:

| Metric | Value |
|:-------|------:|
| Mean | 90 days |
| Median (Q2) | 50 days |
| Std | 99 days |

The histogram shows customer activity drops significantly after 50 days. Median was chosen over mean because:
1. **Robust to outliers:** High std (99) indicates extreme values affecting mean
2. **Data-driven:** 50% of active customers purchase within 50 days
3. **Conservative:** Earlier detection allows proactive retention

**Threshold: 50 days** — customers who haven't purchased in over 50 days are considered at risk of churning.

In [ ]:
# Define churn based on Recency > 50 days
# Churn = 1 if customer hasn't purchased in over 50 days
# Churn = 0 if customer purchased within 50 days

rfm['Churn'] = (rfm['Recency'] > 50).astype(int)

# Check churn distribution  
print(rfm['Churn'].value_counts())
print(f"\nChurn Rate: {rfm['Churn'].mean()*100:.1f}%")

Churn
0    2198
1    2124
Name: count, dtype: int64

Churn Rate: 49.1%


### 5.2 Model Comparison

Four machine learning models were tested to find the best churn predictor.
- Logistic

### 5.2.1 Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Select features and target
X = rfm[['Recency', 'Frequency', 'Monetary']]
y = rfm['Churn']

# Split data: 70% train, 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train Logistic Regression
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

# Predict
y_pred_lr = lr.predict(X_test)

# Evaluate
print("Logistic Regression")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_lr):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

Logistic Regression
Accuracy: 1.0000
F1-Score: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       670
           1       1.00      1.00      1.00       627

    accuracy                           1.00      1297
   macro avg       1.00      1.00      1.00      1297
weighted avg       1.00      1.00      1.00      1297



### Attempt 1 result: 100% accuracy → Data leakage detected

In [ ]:
# check data time range
print(f"First order: {data['InvoiceDate'].min()}")
print(f"Last order: {data['InvoiceDate'].max()}")

First order: 2010-12-01 08:26:00
Last order: 2011-12-09 12:50:00


***Note: Data Leakage Detected***

***The model achieved 100% accuracy because Churn was defined directly from Recency (>50 days), and Recency was used as a feature. This is **data leakage** , the model simply learned the rule we created, not actual customer behavior.***

**Solution:** Split data into two time periods:
- **Training period (Dec 2010 - Sep 2011):** Calculate RFM features
- **Label period (Oct - Dec 2011):** Define Churn based on whether customer purchased

***This ensures the model predicts future behavior, not the same data used to define churn.***

###  Solution: Time-based Split

To avoid data leakage, dataset was split into two periods:

| Period | Date Range | Usage |
|:-------|:-----------|:------|
| Features | Dec 2010 - Sep 2011 | Calculate RFM |
| Label | Oct - Dec 2011 | Define Churn |

In [ ]:
# Time-based Split Visualization
import plotly.graph_objects as go

# Create timeline visualization
fig = go.Figure()

# Training Period (Dec 2010 - Sep 2011)
fig.add_trace(go.Bar(
    x=[9],  # 9 months
    y=['Timeline'],
    orientation='h',
    name='Training Period (Features)',
    marker_color='#4C78A8',
    text=['RFM Features<br>Dec 2010 - Sep 2011'],
    textposition='inside'
))

# Label Period (Oct - Dec 2011)
fig.add_trace(go.Bar(
    x=[3],  # 3 months
    y=['Timeline'],
    orientation='h',
    name='Label Period (Churn)',
    marker_color='#F58518',
    text=['Churn Label<br>Oct - Dec 2011'],
    textposition='inside',
    base=9
))

fig.update_layout(
    title='Time-based Split: Avoiding Data Leakage',
    barmode='stack',
    xaxis_title='Months',
    showlegend=True,
    height=250
)

fig.show()

In [ ]:
# Customers who purchased in Oct-Dec 2011
label_start = '2011-10-01'
active_customers = data[data['InvoiceDate'] >= label_start]['CustomerID'].unique()

# Churn = 1 if NOT purchased in last 3 months
rfm['Churn_Real'] = (~rfm.index.isin(active_customers)).astype(int)

print(rfm['Churn_Real'].value_counts())
print(f"\nChurn Rate: {rfm['Churn_Real'].mean()*100:.1f}%")

Churn_Real
0    2591
1    1731
Name: count, dtype: int64

Churn Rate: 40.1%


### Attempt 2: 68% accuracy → After time-based split (No leakage)


The previous RFM was calculated using the entire dataset (Dec 2010 - Dec 2011), which includes the label period. This still causes data leakage.

**Solution:** Recalculate RFM using only the training period (Dec 2010 - Sep 2011).

In [ ]:
# Filter data for training period only (Dec 2010 - Sep 2011)
train_end = '2011-09-30'
df_train = data[data['InvoiceDate'] <= train_end]

# Snapshot date = first day of label period
snapshot_date = pd.to_datetime('2011-10-01')

# Recalculate RFM from training period only
rfm_new = df_train.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalAmount': 'sum'
})
rfm_new.columns = ['Recency', 'Frequency', 'Monetary']

# Add Churn label (did they purchase in Oct-Dec 2011?)
label_start = '2011-10-01'
active_customers = data[data['InvoiceDate'] >= label_start]['CustomerID'].unique()
rfm_new['Churn'] = (~rfm_new.index.isin(active_customers)).astype(int)

print(f"Customers: {len(rfm_new)}")
print(rfm_new['Churn'].value_counts())
print(f"\nChurn Rate: {rfm_new['Churn'].mean()*100:.1f}%")

Customers: 3646
Churn
0    1881
1    1765
Name: count, dtype: int64

Churn Rate: 48.4%


### Rerun Logistic Regression

In [ ]:
# Train model with rfm_new (no leakage)
X = rfm_new[['Recency', 'Frequency', 'Monetary']]
y = rfm_new['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # use 42 as random state (default seed)

# Logistic Regression
lr = LogisticRegression(random_state=42) 
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

print("Logistic Regression (No Leakage)")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}") # # 4 decimal places
print(f"F1-Score: {f1_score(y_test, y_pred_lr):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

### Logistic Regression Summary

| Metric | Score |
|:-------|------:|
| Accuracy | 68.4% |
| F1-Score | 69.7% |
| Precision (Churn) | 65% |
| Recall (Churn) | 75% |

The model correctly identifies 75% of actual churners (Recall), which is important for proactive retention strategies.

### 5.2.2 Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create Decision Tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Train model with training data
dt.fit(X_train, y_train)

# Predict churn on test data
y_pred_dt = dt.predict(X_test)

# Evaluate model performance
print("Decision Tree")
print(f"Accuracy: {accuracy_score(y_test, y_pred_dt):.4f}") 
print(f"F1-Score: {f1_score(y_test, y_pred_dt):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_dt))

Decision Tree
Accuracy: 0.6143
F1-Score: 0.5911

Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.65      0.63       564
           1       0.61      0.58      0.59       530

    accuracy                           0.61      1094
   macro avg       0.61      0.61      0.61      1094
weighted avg       0.61      0.61      0.61      1094



### Decision Tree Summary

| Metric | Score |
|:-------|------:|
| Accuracy | 61.4% |
| F1-Score | 59.1% |
| Recall (Churn) | 58% |

Decision Tree underperforms Logistic Regression. Likely due to overfitting on training data.

### 5.2.3 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create Random Forest with 100 trees
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train model
rf.fit(X_train, y_train)

# Predict
y_pred_rf = rf.predict(X_test)

# Evaluate
print("Random Forest")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_rf):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

Random Forest
Accuracy: 0.6508
F1-Score: 0.6403

Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.66      0.66       564
           1       0.64      0.64      0.64       530

    accuracy                           0.65      1094
   macro avg       0.65      0.65      0.65      1094
weighted avg       0.65      0.65      0.65      1094



### 5.2.4 Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Create Gradient Boosting with 100 trees
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Train model
gb.fit(X_train, y_train)

# Predict
y_pred_gb = gb.predict(X_test)

# Evaluate
print("Gradient Boosting")
print(f"Accuracy: {accuracy_score(y_test, y_pred_gb):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_gb):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_gb))

Gradient Boosting
Accuracy: 0.6819
F1-Score: 0.6887

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.64      0.67       564
           1       0.65      0.73      0.69       530

    accuracy                           0.68      1094
   macro avg       0.68      0.68      0.68      1094
weighted avg       0.69      0.68      0.68      1094



### Note: F1-Score and Recall over Accuracy

For churn prediction, Recall is more important than Accuracy. Accuracy measures overall correct predictions, while Recall measures how many actual churners were caught by the model.

Missing a churner (especially Super VIP) means direct revenue loss — as shown in Pareto Analysis, losing one Super VIP results in approximately 2% revenue loss. Therefore, it is better to catch more churners even if some predictions are wrong.

Logistic Regression achieved the highest Recall (75%), meaning it correctly identifies 75% of customers who will actually churn. This makes it the best model for proactive retention strategies.

## 5.3 Summary Models
| Model | Accuracy | F1-Score | Recall (Churn) |
|:------|:--------:|:--------:|:--------------:|
| **Logistic Regression** | 68.4% | **69.7%** | **75%** |
| Gradient Boosting | 68.2% | 68.9% | 73% |
| Random Forest | 65.1% | 64.0% | 64% |
| Decision Tree | 61.4% | 59.1% | 58% |

**Best Model: Logistic Regression** — highest F1-Score (69.7%) and Recall (75%), correctly identifying 75% of actual churners for proactive retention strategies.

In [ ]:
# ============================================================
# Export data for Dashboard
# ============================================================

# 1. Cleaned data
df_clean = data.dropna(subset=['CustomerID'])
df_clean = df_clean[df_clean['Quantity'] > 0]
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['UnitPrice']
df_clean.to_csv('../output/cleaned_data.csv', index=False)

# 2. RFM with clusters (K=4)
rfm.to_csv('../output/rfm_clusters.csv')

# 3. RFM for churn prediction (time-based split)
rfm_new.to_csv('../output/rfm_churn.csv')

print("✅ Exported successfully!")

✅ Exported successfully!
